In [1]:
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F
import spacy
!jupyter nbextension enable --py widgetsnbextension
from tqdm.notebook import tqdm
from torchtext.data.metrics import bleu_score
from torchtext.data import Field, BucketIterator,TabularDataset
from torchtext.data.utils import get_tokenizer
import pandas as pd
from sklearn.model_selection import train_test_split
# CUDA=False
CUDA=torch.cuda.is_available()
device=torch.device("cuda:7" if CUDA else "cpu")

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [2]:
with open('/mnt/disk1/Gulshan/rnn/translation/data.en1','r') as f:
    eng=f.readlines()
with open('/mnt/disk1/Gulshan/rnn/translation/data.ta1','r') as f:
    tam=f.readlines()
# a=open('/mnt/disk1/Gulshan/rnn/translation/data.en1','r')

In [3]:
# e1=[i.strip().split(' ') for i in tam]
# e2=[i.strip().split(' ') for i in eng]
# e1[0],e2[0]
def tokenize_(sen):
    return [word.strip() for word in sen.split(' ')]

In [4]:
tamil_token=Field(tokenize=tokenize_, lower=True, init_token="<sos>", eos_token="<eos>")


In [5]:
tamil_txt=open('/mnt/disk1/Gulshan/rnn/translation/data.ta1',encoding='utf8').read().split('\n')
english_txt=open('/mnt/disk1/Gulshan/rnn/translation/data.en1',encoding='utf8').read().split('\n')

In [6]:
raw_data={'English':[line for line in english_txt],
          'Tamil':[line for line in tamil_txt]}

df=pd.DataFrame(raw_data,columns=['English','Tamil'])
df.head(2)

,English,Tamil
0,"moreover all the vessels , which king ahaz in ...",ராஜாவாகிய ஆகாஸ் அரசாளும்போது தம்முடைய பாதகத்தி...
1,similar conditions will be imposed if the sri ...,சர்வதேச நாணய நிதியம் இலங்கைக்கு கடன் வழங்கினால...


In [7]:
train_val,test=train_test_split(df,test_size=0.2)
train,val=train_test_split(train_val,test_size=0.2)
len(train_val),len(test),len(train),len(val)

(40000, 10001, 32000, 8000)

In [8]:
train.to_json('train_ml_translat.json',orient='records',lines=True)
val.to_json('val_ml_translat.json',orient='records',lines=True)
test.to_json('test_ml_translat.json',orient='records',lines=True)

In [9]:
def tokenize_(text):
    return [word for word in text.split(' ')]

In [10]:
english=Field(tokenize=tokenize_, lower=True, init_token="<start>", eos_token="<end>")
tamil=Field(tokenize=tokenize_, lower=True, init_token="<start>", eos_token="<end>")

In [11]:
fields={'English':('eng',english),
        'Tamil':('tam',tamil)}
# fields={'English':english,
#         'Tamil':tamil}

In [12]:
train_data,val_data,test_data=TabularDataset.splits(
    path='',
    train='/mnt/disk1/Gulshan/rnn/translation/train_ml_translat.json',
    validation='/mnt/disk1/Gulshan/rnn/translation/val_ml_translat.json',
    test='/mnt/disk1/Gulshan/rnn/translation/test_ml_translat.json',
    format='json',
    fields=fields
)

In [13]:
english.build_vocab(train_data,max_size=10000,min_freq=2)
tamil.build_vocab(train_data,max_size=10000,min_freq=2)

In [14]:
tamil.vocab.stoi,english.vocab.stoi

(defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7fb4c2139c70>>,
             {'<unk>': 0,
              '<pad>': 1,
              '<start>': 2,
              '<end>': 3,
              '.': 4,
              ',': 5,
              'ஒரு': 6,
              'மற்றும்': 7,
              'என்று': 8,
              'இந்த': 9,
              'நான்': 10,
              '': 11,
              'அவர்': 12,
              'அமெரிக்க': 13,
              '?': 14,
              'வேண்டும்': 15,
              'அது': 16,
              'அவர்கள்': 17,
              'என்ற': 18,
              'என': 19,
              'அரசியல்': 20,
              'இது': 21,
              'இருந்து': 22,
              'ஆனால்': 23,
              'நீங்கள்': 24,
              'அந்த': 25,
              'என்': 26,
              'இருக்கும்': 27,
              'அதன்': 28,
              'பற்றி': 29,
              'பல': 30,
              'உள்ள': 31,
              'இல்லை': 32,
              'அல்லது': 33,
 

In [15]:
# train_loader,val_loader,test_loader=BucketIterator.splits((train_data,val_data,test_data),
#                                                batch_size=8,
#                                                device=device,
#                                                )

In [16]:
# batch=next(iter(train_loader))
# print(type(batch))
# batch.tam.shape, batch.eng.shape

In [17]:
# a

In [18]:
torch.save({
                'tamil_voc': tamil.vocab.__dict__,
                'english_voc': english.vocab.__dict__,
            },'vocab_tamil_english.pth.tar')

# transformer

In [19]:
class _transfomer(nn.Module):
    def __init__(self,embed_size,src_vocab_size,trg_vocab_size,src_pad_idx,
                 num_heads,n_enc_layer,n_dec_layer,forward_expansion,
                 dropout,max_len,device):
        super(_transfomer, self).__init__()
        self.src_embedding=nn.Embedding(src_vocab_size,embed_size) 
        self.src_postion_embedding=nn.Embedding(max_len,embed_size) 
        self.trg_embedding=nn.Embedding(trg_vocab_size,embed_size) 
        self.trg_postion_embedding=nn.Embedding(max_len,embed_size)
        self.device=device 
        self.tranformer=nn.Transformer(embed_size,
                                       num_heads,
                                       n_enc_layer,
                                       n_dec_layer,
                                       forward_expansion,
                                       dropout,
                                       )
        self.fc_out=nn.Linear(embed_size,trg_vocab_size)
        self.dropout=nn.Dropout(dropout)
        self.src_pad_idx=src_pad_idx
        
    def _make_src_mask(self,src):
        src_mask=src.transpose(0,1)==self.src_pad_idx
        return src_mask
    
    def forward(self,src,trg):
        src_seq_len,N=src.shape
        trg_seq_len,N=trg.shape
        
        src_position=(
            torch.arange(0,src_seq_len).unsqueeze(1).expand(src_seq_len,N).to(self.device)
        )
        trg_position=(
            torch.arange(0,trg_seq_len).unsqueeze(1).expand(trg_seq_len,N).to(self.device)
        )
        
        embed_src=self.dropout(
            (self.src_embedding(src) + self.src_postion_embedding(src_position))
            )
        
        embed_trg=self.dropout(
            (self.trg_embedding(trg) + self.trg_postion_embedding(trg_position))
        )
        _src_mask_pad=self._make_src_mask(src)
        trg_mask=self.tranformer.generate_square_subsequent_mask(trg_seq_len).to(self.device)

        out=self.tranformer(
            embed_src,
            embed_trg,
            src_key_padding_mask=_src_mask_pad,
            tgt_mask=trg_mask
        )
        out=self.fc_out(out)
        
        return out
        
    

# Hpyerparameterds

In [20]:
src_vocab_size=len(english.vocab)
trg_vocab_size=len(tamil.vocab)
num_epochs=100
lr=20**-4
bs=32
# embed_size=512
# num_head=8
embed_size=768
num_head=12
n_ecode_layer=8 #6
n_decode_layer=8
dropout=0.2#2
max_len=100
forward_expansion=4
src_pad_idx=english.vocab.stoi["<pad>"]
writer=SummaryWriter("runs/loss_plot")
pad_idx=english.vocab.stoi["<pad>"]

In [21]:
train_loader,val_loader,test_loader=BucketIterator.splits((train_data,val_data,test_data),
                                               batch_size=bs,
                                               sort_within_batch=True,
                                               sort_key= lambda x:len(x.eng), 
                                               device=device
                                               )

In [22]:
model=_transfomer(embed_size,src_vocab_size,trg_vocab_size,
                  src_pad_idx,num_head,n_ecode_layer,n_decode_layer,
                  forward_expansion,dropout,max_len,device).to(device)
optimizer=torch.optim.Adam(model.parameters(),lr=lr)
criterion=nn.CrossEntropyLoss(ignore_index=pad_idx)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, factor=0.1, patience=10, verbose=True
)

/mnt/disk1/conda/envs/gulshan/lib/python3.8/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
/mnt/disk1/conda/envs/gulshan/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [23]:
load_model=True
if load_model:
    model = _transfomer(embed_size,src_vocab_size,trg_vocab_size,
                  src_pad_idx,num_head,n_ecode_layer,n_decode_layer,
                  forward_expansion,dropout,max_len,device).to(device)
    _load=torch.load('/mnt/disk1/Gulshan/rnn/translation/checkpoints/6.25e-06_768_12_100.pth.tar')
    _load,op,ep,sh=_load['model'],_load['optimizer'],_load['epoch'],_load['lr_scheduler']
    model.load_state_dict(_load)
    optimizer.load_state_dict(op)
    scheduler.load_state_dict(sh)
    epochs=ep
epochs=50
    

/mnt/disk1/conda/envs/gulshan/lib/python3.8/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


# training

In [24]:
def train():
    step=0
    for epoch in tqdm(range(epochs,num_epochs), total=num_epochs):
        
        model.train()
        losses = []
        for batch_idx,batch in enumerate(tqdm(train_loader)):
            src_data=batch.eng.to(device)
            trg_data=batch.tam.to(device)
            # forward
            #trg_data[:-1] -> hey ! good  <end> , src data -> <start> hey ! good
            out=model(src_data,trg_data[:-1])
            out=out.reshape(-1,out.shape[2]) # to keep the last dim 
            trg_data=trg_data[1:].reshape(-1)
            
            optimizer.zero_grad()
            loss=criterion(out,trg_data)
            losses.append(loss.item())
            loss.backward()
            
            nn.utils.clip_grad_norm_(model.parameters(),max_norm=1)
            
            optimizer.step()
            
            writer.add_scalar( "Training loss",loss,global_step=step)
            step+=1
        mean_loss = sum(losses) / len(losses)
        scheduler.step(mean_loss)
        print("The mean loss is",mean_loss)
        if (epoch+1)%5==0:
            print(1)
            state = {'epoch': epoch+1, 'model': model.state_dict(),
                     'optimizer': optimizer.state_dict(),
                     'lr_scheduler':scheduler.state_dict()}
            torch.save(state, f'/mnt/disk1/Gulshan/rnn/translation/checkpoints/{lr}_{embed_size}_{num_head}_' + str(epoch+1) + '.pth.tar')
train()
# 4.233643220424652


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

The mean loss is 5.781579265594482


  0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# 4.233643220424652

In [ ]:
_model = _transfomer(embed_size,src_vocab_size,trg_vocab_size,
                  src_pad_idx,num_head,n_ecode_layer,n_decode_layer,
                  forward_expansion,dropout,max_len,device).to(device)
_load=torch.load('/mnt/disk1/Gulshan/rnn/translation/checkpoints/6.25e-06_checkpoint_40.pth.tar')
_load=_load['transformer']
_model.load_state_dict(_load)
# _model

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/disk1/Gulshan/rnn/translation/checkpoints/6.25e-06_checkpoint_40.pth.tar'

In [ ]:
def translate_sentence(_model, sentence, tamil, english, device, max_length=50):
    _model.eval()
    if type(sentence) == str:
        tokens = [token.lower() for token in sentence.split(' ')]
    else:
        tokens = [token.lower() for token in sentence]
    tokens.insert(0, english.init_token)
    tokens.append(english.eos_token)
    text_to_indices = [english.vocab.stoi[token] for token in tokens]
    print(text_to_indices)
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    outputs = [tamil.vocab.stoi["<start>"]]
    for i in range(max_length):
        trg_tensor = torch.LongTensor(outputs).unsqueeze(1).to(device)

        with torch.no_grad():
            output = _model(sentence_tensor, trg_tensor)

        best_guess = output.argmax(2)[-1, :].item()

        outputs.append(best_guess)
        if best_guess == tamil.vocab.stoi["<end>"]:
            break

    translated_sentence = [tamil.vocab.itos[idx] for idx in outputs]
    # remove start token
    print(translated_sentence[1:-1])
    output=' '.join([(i) for i in translated_sentence[1:-1]])
    return output

In [ ]:
english.init_token

'<start>'

In [ ]:
sen="come here"
translate_sentence(_model,sen,tamil,english,device)

[2, 122, 235, 3]
['நிக்கோலா', 'பேர்லின்', 'பிரச்சினைகளின்', 'செய்யப்படவில்லை', 'வார்த்தைகளில்', 'இருந்தார்', 'பேர்லின்', 'தேடிக்', 'ஊழியர்கள்', 'ரோகல்', 'வார்த்தைகளில்', 'பிரச்சினைகளின்', 'முதலாக', 'பகுதிகள்', 'சொல்லுகிறார்;', 'பூமி', 'மற்றவர்கள்', 'பூசலில்', 'பேர்லின்', 'தெரிகிறது', 'நண்பர்', 'தேடிக்', 'சொல்லுகிறார்;', 'தேடிக்', 'தேடிக்', 'கழுத்தை', 'முன்னெடுக்கும்', 'குமாரதுங்கவின்', 'சொல்லுகிறார்;', 'உனக்கு', 'தேடிக்', 'கூறியதாவது', 'தேடிக்', 'அதேபோல்', 'பெற்றுள்ளார்', 'தேடிக்', 'பேர்லின்', 'தேடிக்', 'இருந்தார்', 'ரஷ்யாவுடன்', 'கூறியதாவது', 'பேர்லின்', 'கூறியதாவது', 'வார்த்தை', 'ஊழியர்கள்', 'பிரச்சினைகளின்', 'வார்த்தைகளில்', 'வார்த்தைகளில்', 'தேடிக்']


'நிக்கோலா பேர்லின் பிரச்சினைகளின் செய்யப்படவில்லை வார்த்தைகளில் இருந்தார் பேர்லின் தேடிக் ஊழியர்கள் ரோகல் வார்த்தைகளில் பிரச்சினைகளின் முதலாக பகுதிகள் சொல்லுகிறார்; பூமி மற்றவர்கள் பூசலில் பேர்லின் தெரிகிறது நண்பர் தேடிக் சொல்லுகிறார்; தேடிக் தேடிக் கழுத்தை முன்னெடுக்கும் குமாரதுங்கவின் சொல்லுகிறார்; உனக்கு தேடிக் கூறியதாவது தேடிக் அதேபோல் பெற்றுள்ளார் தேடிக் பேர்லின் தேடிக் இருந்தார் ரஷ்யாவுடன் கூறியதாவது பேர்லின் கூறியதாவது வார்த்தை ஊழியர்கள் பிரச்சினைகளின் வார்த்தைகளில் வார்த்தைகளில் தேடிக்'

In [ ]:
def bleu(data, model, tamil, english, device):
    targets = []
    outputs = []

    for example in data:
        src = vars(example)["src"]
        trg = vars(example)["trg"]

        prediction = translate_sentence(model, src, tamil, english, device)
        prediction = prediction[:-1]  # remove <eos> token

        targets.append([trg])
        outputs.append(prediction)

    return bleu_score(outputs, targets)